In [18]:
# 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
# 2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
# 3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [41]:
from pymongo import MongoClient
import pandas as pd
import json
from pprint import pprint

In [44]:
client = MongoClient('localhost', 27017)
db = client['PSC_3']
jobs_hh = db.jobs_hh
jobs_sj = db.jobs_sj

In [45]:
df_hh = pd.read_csv('job_search_hh.csv')
df_sj = pd.read_csv('job_search_sj.csv')

In [60]:
df_hh.head()

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,HeadHunter::Технический департамент,Big Data Engineer,250000.0,NaN,Москва,Алексеевская,Уверенные знания принципов работы баз данных и...,"Создание ETL-процессов, используя внутренние и...",23 сентября,https://hh.ru/vacancy/32182817?query=Data%20sc...,hh.ru
1,ООО Диджитал медикэл оперейшнс,Senior Data Scientist / NLP инженер,180000.0,215000.0,Москва,NaN,Хорошие навыки в области классического NLP (pr...,Разработка системы анализа/понимания неструкту...,24 сентября,https://hh.ru/vacancy/33458182?query=Data%20sc...,hh.ru
2,BestDoctor,Data Scientist,NaN,NaN,Москва,Парк Победы,Хорошие знания математической статистики и тео...,Модель скоринга. Прогнозирование финансовых ри...,24 сентября,https://hh.ru/vacancy/33282290?query=Data%20sc...,hh.ru
3,Сбербанк для экспертов,Руководитель направления по исследованию данны...,NaN,NaN,Москва,NaN,Data Engineers с опытом обработки потоковых да...,Моделирование: Feature Enginering: - методы оц...,24 сентября,https://hh.ru/vacancy/33747160?query=Data%20sc...,hh.ru
4,Skillbox,Data Scientist,NaN,NaN,Москва,NaN,Опыт работы с python 3.6+ (со стеком библиотек...,Проверка качества данных. Написание скриптов д...,24 сентября,https://hh.ru/vacancy/32742065?query=Data%20sc...,hh.ru


In [61]:
df_sj.head()

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,Business connection,"Программист С++, Golang, PHP, Python, DevOPS (...",100000,250000.0,Москва,NaN,Знание языка программирования C++ на продвинут...,О компании. Написание с нуля без использования...,вчера,https://www.superjob.ru/clients/business-conne...,superjob.ru
1,НТИМИ,Программист на python,100000,250000.0,Москва,NaN,"Опыт программирования на Python от 2 лет, базо...",Вам предстоит разрабатывать автоматизированную...,20 сентября,https://www.superjob.ru/vakansii/programmist-n...,superjob.ru
2,ВНИИМС,Аналитик (инженер-математик),80000,NaN,Москва,Озёрная,"Высшее образование в области математики, техни...",Разработка прогнозных моделей в части оценки п...,вчера,https://www.superjob.ru/vakansii/analitik-3256...,superjob.ru
3,Тинькофф,Специалист Отдела Операционного планирования и...,50000,NaN,Москва,NaN,"Логическое, абстрактное мышление, умение сопос...",Сбор и анализ информации по работе подразделен...,вчера,https://www.superjob.ru/clients/tinkoff-160502...,superjob.ru
4,МобильныеТелеСистемы,Старший инженер эксплуатации ТВ платформ,50000,NaN,Москва,Пражская,NaN,Образование высшее в области связи и радиотехн...,вчера,https://www.superjob.ru/clients/mts-81084.html,superjob.ru


In [46]:
# Inserts pandas dataframe to colletion
def df_to_db(df, db_collection):
    data = json.loads(df.T.to_json()).values()
    db_collection.insert_many(data)

df_to_db(df_hh, jobs_hh)
df_to_db(df_sj, jobs_sj)

In [57]:
amount = 100000
func = 'gt'

params = {'firm_name', 'vacancy_name', 'salary_min', 'salary_max', 'city',
       'metro_station', 'prereq_shrt', 'tasks_shrt', 'publish_date',
       'vacancy_link', 'site'}


def show_search_results(db, amount, func, params):

    n = db.find({'salary_min':{f'${func}':amount}}, params)
    for item in n:
        pprint(item)


In [58]:
show_search_results(jobs_hh, gt_amount, func, params)

{'_id': ObjectId('5d8a8a722e73b8eefa63dfa2'),
 'city': 'Москва ',
 'firm_name': ' HeadHunter::Технический департамент',
 'metro_station': 'Алексеевская',
 'prereq_shrt': 'Уверенные знания принципов работы баз данных и распределенных '
                'систем хранения и обработки данных. Опыт работы с '
                'распределенными системами хранения и...',
 'publish_date': '23 сентября',
 'salary_max': None,
 'salary_min': 250000.0,
 'site': 'hh.ru',
 'tasks_shrt': 'Создание ETL-процессов, используя внутренние и внешние '
               'источники данных. Разработка и поддержка инфраструктуры для '
               'хранения и обработки данных. ',
 'vacancy_link': 'https://hh.ru/vacancy/32182817?query=Data%20scientist',
 'vacancy_name': 'Big Data Engineer'}
{'_id': ObjectId('5d8a8a722e73b8eefa63dfa3'),
 'city': 'Москва',
 'firm_name': 'ООО Диджитал медикэл оперейшнс',
 'metro_station': None,
 'prereq_shrt': 'Хорошие навыки в области классического NLP (preprocessing, '
               

In [59]:
show_search_results(jobs_sj, amount, func, params)

{'_id': ObjectId('5d8a8a722e73b8eefa63e081'),
 'city': 'Москва',
 'firm_name': 'МобильныеТелеСистемы',
 'metro_station': None,
 'prereq_shrt': 'Образование высшее в области связи и радиотехники. Знание '
                'работы сетевых служб и протоколов '
                '(http/smtp/smb/ftp/ssh/telne…',
 'publish_date': 'вчера',
 'salary_max': 110000.0,
 'salary_min': 110000,
 'site': 'superjob.ru',
 'tasks_shrt': 'Администрирование, конфигурация и поддержка сетевого '
               'оборудования, серверного оборудования, СХД, ОС Linux, СУБД в '
               'составе…',
 'vacancy_link': 'https://www.superjob.ru/clients/mts-81084.html',
 'vacancy_name': 'Ведущий инженер Команда МТС ТВ'}
{'_id': ObjectId('5d8a8a722e73b8eefa63e082'),
 'city': 'Москва',
 'firm_name': 'SuperJob',
 'metro_station': 'Маяковская  Новослободская\xa0и ещё 1 станция ',
 'prereq_shrt': 'Высокий уровень знания Google analytics, Яндекс. Метрика, '
                'Excel. Опыт работы в любой BI-системе (у нас Tab